In [5]:
# !pip install fastf1

In [1]:
import fastf1
import pandas as pd
import numpy as np

fastf1.Cache.enable_cache('cache')

# Loading data

In [2]:
calendar20 = fastf1.get_event_schedule(2020)
calendar21 = fastf1.get_event_schedule(2021)
calendar22 = fastf1.get_event_schedule(2022)
calendar23 = fastf1.get_event_schedule(2023)
calendar24 = fastf1.get_event_schedule(2024)

In [4]:
calendar20 = calendar20[['RoundNumber', 'EventName']]
calendar21 = calendar21[['RoundNumber', 'EventName']]
calendar22 = calendar22[['RoundNumber', 'EventName']]
calendar23 = calendar23[['RoundNumber', 'EventName']]
calendar24 = calendar24[['RoundNumber', 'EventName']]

calendar20.loc[:, 'Season'] = 2020
calendar21.loc[:, 'Season'] = 2021
calendar22.loc[:, 'Season'] = 2022
calendar23.loc[:, 'Season'] = 2023
calendar24.loc[:, 'Season'] = 2024

In [5]:
calendar = pd.concat([
    calendar20,
    calendar21,
    calendar22,
    calendar23,
    calendar24], ignore_index=True)

In [6]:
calendar

,RoundNumber,EventName,Season
0,0,Pre-Season Test 1,2020
1,0,Pre-Season Test 2,2020
2,1,Austrian Grand Prix,2020
3,2,Styrian Grand Prix,2020
4,3,Hungarian Grand Prix,2020
...,...,...,...
109,20,Mexico City Grand Prix,2024
110,21,São Paulo Grand Prix,2024
111,22,Las Vegas Grand Prix,2024
112,23,Qatar Grand Prix,2024


In [7]:
all_laps = []
all_track_status = []
all_events = []

for season in range(2020, 2025):
    calendar = fastf1.get_event_schedule(season)

    for _, row in calendar.iterrows():
        session = fastf1.get_session(season, row['RoundNumber'], 'R')

        session.load(
            telemetry=False,
            messages=False
        )

        # Event data
        all_events.append({
            'Season': season,
            'RoundNumber': row['RoundNumber'],
            'Race': row['EventName'],
            'Location': session.event['Location'],
            'Country': session.event['Country'],
            'Circuit': session.event['OfficialName']
        })

        # Laps
        laps = session.laps.copy()
        laps['Season'] = season
        laps['RoundNumber'] = row['RoundNumber']
        laps['Race'] = row['EventName']
        all_laps.append(laps)

        # Track status
        if session.track_status is not None:
            ts = session.track_status.copy()
            ts['Season'] = season
            ts['RoundNumber'] = row['RoundNumber']
            ts['Race'] = row['EventName']
            all_track_status.append(ts)

logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	No cached data found for season_schedule. Loading data...
_api           INFO 	Fetching season schedule...
req            INFO 	Data has been written to cache!
core           INFO 	Loading data for Austrian Grand Prix - Race [v3.7.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            I

KeyError: 'OfficialName'

In [14]:
session = fastf1.get_session(2023, 'Bahrain', 'R')

In [15]:
session.load(telemetry=False, messages=False)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for weather_data
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']


In [32]:
[attr for attr in dir(session) if not attr.startswith('_')]

['api_path',
 'car_data',
 'date',
 'drivers',
 'event',
 'f1_api_support',
 'get_circuit_info',
 'get_driver',
 'laps',
 'load',
 'name',
 'pos_data',
 'race_control_messages',
 'results',
 'session_info',
 'session_start_time',
 'session_status',
 't0_date',
 'total_laps',
 'track_status',
 'weather_data']